# Model with a simplified sigmoid scale

[Index](../0-index.ipynb)

Here we investigate whether a model with a sigmoid scale $p \beta$ is a good approximation of the pandemics spreading.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from pathlib import Path
import os,sys
import numpy as np
import pandas as pd
import datetime

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import matplotlib.colors as mco
import matplotlib.gridspec as mgs
import matplotlib.cm as cm
from matplotlib import animation
plt.rcParams['svg.fonttype'] = 'none'

from IPython.display import HTML
from IPython.display import Image

In [ ]:
sys.path.append(str(Path('../..') / 'code'))

In [ ]:
resdir = Path('../../results/')
if not resdir.is_dir():
    raise ValueError('No results directory!')

In [ ]:
resfile = resdir / 'safegraph_analysis.hdf5'
complevel=7
complib='zlib'
with pd.HDFStore(resfile, complevel=complevel, complib=complib) as store:
    print(f"File {resfile.stem} has {len(store.keys())} entries.")

## Global variables and other quantities

### Global variables

In [ ]:
gamma = 1/10.
ti = '2020-03-01'
tf = '2021-02-16'

tfmt = '%Y-%m-%d'
ti = datetime.datetime.strptime(ti, tfmt)
tf = datetime.datetime.strptime(tf, tfmt)

pathtofit = Path('/fit')
pathtosimu = Path('/simulations/simplified')

exts = ['.png', '.svg']

### Load clusters to get population

In [ ]:
key = "/clustering/clusters"
with pd.HDFStore(resfile, complevel=complevel, complib=complib) as store:
    clusters = store[key]
# clusters = pd.read_hdf(resfile, key)
N = len(clusters)
print(f"N = {N}")
clusters

In [ ]:
population = clusters['population'].to_numpy()
population_inv = np.zeros(population.shape, dtype=np.float_)
idx = population > 0.
population_inv[idx] = 1./population[idx]

### Load CSSEGI data

In [ ]:
path = '/clustering/cssegi'
with pd.HDFStore(resfile, complevel=complevel, complib=complib) as store:
    df_cssegi = store[path]

times = df_cssegi.index
idx = (times >= ti) & (times <= tf)
df_cssegi.drop(index=times[~idx], inplace=True)
times = df_cssegi.index.to_pydatetime().tolist()
df_cssegi

In [ ]:
omega_real = df_cssegi.to_numpy().astype('float64')
domega_real = np.diff(omega_real, axis=0)
domega_real = np.concatenate([omega_real[0].reshape(1,-1), domega_real], axis=0)

In [ ]:
# compute the real epidemic sizes per community through time
T_real = np.einsum('ta,a->ta', omega_real, population_inv)
dT_real = np.einsum('ta,a->ta', domega_real, population_inv)
T_tot_real = np.einsum('ta,a->t', T_real, population) / np.sum(population)
dT_tot_real = np.einsum('ta,a->t', dT_real, population) / np.sum(population)

### Read fit

In [ ]:
with pd.HDFStore(resfile, complevel=complevel, complib=complib) as store:
    mykey = str(pathtofit / 'result' / 'fit')
    df_fit = store[mykey]

times = df_fit.index
idx = (times >= ti) & (times <= tf)
df_fit.drop(index=times[~idx], inplace=True)
times = df_fit.index.to_pydatetime().tolist()
df_fit

### Determine lockdown time

In [ ]:
b_scales = df_fit['scale_step'].fillna(value=np.nan).to_numpy()
ic = np.nanargmax(np.abs(np.diff(b_scales))) + 1
tc = df_fit.index[ic]
print(f"lockdown at t = {tc}")

In [ ]:
df_fit.iloc[ic-1:ic+2]

## Perform simulation

### Construct the localization matrices

In [ ]:
pathtoloc = pathtosimu / 'infectivity_matrices'

In [ ]:
times = df_fit.index

In [ ]:
with pd.HDFStore(resfile, complevel=complevel, complib=complib) as store:
    key = pathtofit / 'infectivity_matrices' / times[0].strftime(tfmt)
    key = str(key)
    df_loc = store[key]

with pd.HDFStore(resfile, complevel=complevel, complib=complib) as store:
    t = times[0]
    pref = t.strftime(tfmt)

    key = pathtoloc / pref
    key = str(key)
    store[key] = df_loc

### Set initial condition

In [ ]:
from functions import sir_SI_to_X

In [ ]:
with pd.HDFStore(resfile, complevel=complevel, complib=complib) as store:
    mykey = str(pathtofit / 'result' / 'susceptible')
    df_S_fit = store[mykey]
    
    mykey = str(pathtofit / 'result' / 'infected')
    df_I_fit = store[mykey]

In [ ]:
Si = df_S_fit.iloc[0]
Ii = df_I_fit.iloc[0]
Xi = sir_SI_to_X(Si, Ii)

In [ ]:
T_fit = 1. - df_S_fit.to_numpy().astype('float64')
dT_fit = np.diff(T_fit, axis=0)
dT_fit = np.concatenate([T_fit[0].reshape(1,-1), dT_fit], axis=0)

df_T_fit = pd.DataFrame(data=T_fit, index=df_S_fit.index, columns=df_S_fit.columns)
df_dT_fit = pd.DataFrame(data=dT_fit, index=df_S_fit.index, columns=df_S_fit.columns)

In [ ]:
T_tot_fit = np.einsum('ta,a', T_fit, population) / np.sum(population)
dT_tot_fit = np.einsum('ta,a', dT_fit, population) / np.sum(population)

### Simulation

In [ ]:
from functions import integrate_sir, get_sir_omega_SI

In [ ]:
scales = df_fit.loc[:times[-1]-datetime.timedelta(days=1), 'scale_sigmoid'].to_numpy()
with pd.HDFStore(resfile, complevel=complevel, complib=complib) as store:
    ts, Ss, Is  = integrate_sir(Xi, times, scales, gamma, store, pathtoloc)

df_S = pd.DataFrame(data=Ss, index=ts, columns=clusters.index)
df_I = pd.DataFrame(data=Is, index=ts, columns=clusters.index)

path = pathtosimu / 'result'
with pd.HDFStore(resfile, complevel=complevel, complib=complib) as store:
    key = path / 'susceptible'
    store[str(key)] = df_S
    
    key = path / 'infected'
    store[str(key)] = df_I

In [ ]:
path = pathtosimu / 'result'
with pd.HDFStore(resfile, complevel=complevel, complib=complib) as store:
    key = path / 'susceptible'
    df_S = store[str(key)]
    
    key = path / 'infected'
    df_I = store[str(key)]
    
    ts = df_S.index

In [ ]:
S = df_S.to_numpy().astype('float64')
T = (1. - S)
dT = np.diff(T, axis=0)
dT = np.concatenate([T[0].reshape(1,-1), dT], axis=0)

df_T = pd.DataFrame(data=T, index=df_S.index, columns=df_S.columns)
df_dT = pd.DataFrame(data=dT, index=df_S.index, columns=df_S.columns)

In [ ]:
T_tot = np.einsum('ta,a', T, population) / np.sum(population)
dT_tot = np.einsum('ta,a', dT, population) / np.sum(population)

### Plot epidemic size evolution

In [ ]:
figdir = Path('..') / '..' / 'figures' / '6-simulations' / '61-simplified_model'
if not figdir.is_dir():
    figdir.mkdir(parents=True, exist_ok=True)

In [ ]:
# parameters
figsize = (6,4.5)
dpi = 300
ms=2
lw=1
show_dT=True
Z = np.sum(population) / 1000


fig = plt.figure(facecolor='w', figsize=figsize)
ax = fig.gca()

if show_dT:
    ax.plot(times,dT_tot_fit*Z, '-', ms=ms, color='darkblue')
    ax.plot(times,dT_tot*Z, '-', ms=ms, color='darkgreen')
    ax.plot(times,dT_tot_real*Z, 'o', lw=lw, color='red')
    ax.set_ylabel("$d T$", fontsize="medium")
    fname = 'domega_tot_fit'
else:
    ax.plot(times,T_tot_fit*Z, '-', ms=ms, color='darkblue')
    ax.plot(times,T_tot*Z, '-', ms=ms, color='darkgreen')
    ax.plot(times,T_tot_real*Z, 'o', lw=lw, color='red')
    ax.set_ylabel("$T$", fontsize="medium")
    ax.set_yscale('log')
    fname = 'omega_tot_fit'

ax.set_xlim(times[0],None)
plt.xticks(rotation=45)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.tick_params(left=True, labelleft=True, bottom=True, labelbottom=True)
ax.tick_params(axis='both', length=4)
fig.tight_layout()

for ext in exts:
    filepath = figdir / (fname + ext)
    fig.savefig(filepath, bbox_inches='tight', pad_inches=0, dpi=dpi)
    print("Written file: {:s}".format(str(filepath)))
fig.clf()
plt.close('all')

In [ ]:
filepath = figdir / (fname + '.png')
Image(filename=filepath, width=4./3*360)

## Show spatial $T$ agreement

Show $T_a$

In [ ]:
# parameters
vmin = 1.0e0
vmax = 1.0e6
cmap = cm.rainbow
figsize=(4,3)
dpi=300

## color mapping with date value
indices = np.arange(len(ts))
norm = mco.Normalize(vmin=np.min(indices), vmax=np.max(indices))
colors = cmap(norm(indices))

## make figure
fig = plt.figure(facecolor='w', figsize=figsize, dpi=dpi)
ax = fig.gca()

for i in range(len(ts)):
    t = ts[i]
    X = T_fit[i]*population
    Y = T[i]*population
    
    ax.plot(X, Y, 'o', color=colors[i], lw=0, mew=0, ms=2, alpha=0.1)    
ax.plot([vmin, vmax], [vmin, vmax], 'k-', lw=0.5)

# plot formatting
ax.set_xlabel("$T_a^\mathrm{fit}$", fontsize='medium')
ax.set_ylabel("$T_a$", fontsize='medium')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.tick_params(bottom=True, left=True, labelbottom=True, labelleft=True)
ax.tick_params(length=4)
ax.set_xlim(vmin, vmax)
ax.set_ylim(vmin, vmax)
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_aspect('equal')
    
fig.tight_layout(rect=[0.,0.,0.95,1.])
cax = fig.add_axes(rect=[0.99,0.2,0.01,0.7])
cbar = plt.colorbar(cm.ScalarMappable(norm=norm, cmap=cmap),cax=cax, extendfrac='auto')
ticks = cbar.get_ticks()
labels = [ts[int(i)].strftime('%Y-%m-%d') for i in ticks]
cbar.set_ticks(ticks)
cbar.set_ticklabels(labels)

fname = 'T_model_vs_simplified'
for ext in exts:
    filepath = figdir / (fname + ext)
    fig.savefig(filepath, bbox_inches='tight', pad_inches=0, dpi=dpi)
    print("Written file: {:s}".format(str(filepath)))
fig.clf()
plt.close('all')

In [ ]:
filepath = figdir / (fname + '.png')
Image(filename=filepath, width=4./3*360)

Show $dT_a$

In [ ]:
# parameters
vmin = 1.0e0
vmax = 1.0e4
cmap = cm.rainbow
figsize=(4,3)
dpi=300

## color mapping with date value
indices = np.arange(len(ts))
norm = mco.Normalize(vmin=np.min(indices), vmax=np.max(indices))
colors = cmap(norm(indices))

## make figure
fig = plt.figure(facecolor='w', figsize=figsize, dpi=dpi)
ax = fig.gca()

for i in range(len(ts)):
    t = ts[i]
    X = dT_fit[i]*population
    Y = dT[i]*population
    
    ax.plot(X, Y, 'o', color=colors[i], lw=0, mew=0, ms=2, alpha=0.1)    

ax.plot([vmin, vmax], [vmin, vmax], 'k-', lw=0.5)
# plot formatting
ax.set_xlabel("$dT_a^\mathrm{fit}$", fontsize='medium')
ax.set_ylabel("$dT_a$", fontsize='medium')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.tick_params(bottom=True, left=True, labelbottom=True, labelleft=True)
ax.tick_params(length=4)
ax.set_xlim(vmin, vmax)
ax.set_ylim(vmin, vmax)
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_aspect('equal')
    
fig.tight_layout(rect=[0.,0.,0.95,1.])
cax = fig.add_axes(rect=[0.99,0.2,0.01,0.7])
cbar = plt.colorbar(cm.ScalarMappable(norm=norm, cmap=cmap),cax=cax, extendfrac='auto')
ticks = cbar.get_ticks()
labels = [ts[int(i)].strftime('%Y-%m-%d') for i in ticks]
cbar.set_ticks(ticks)
cbar.set_ticklabels(labels)


fname = 'dT_model_vs_simplified'
for ext in exts:
    filepath = figdir / (fname + ext)
    fig.savefig(filepath, bbox_inches='tight', pad_inches=0, dpi=dpi)
    print("Written file: {:s}".format(str(filepath)))
fig.clf()
plt.close('all')

In [ ]:
filepath = figdir / (fname + '.png')
Image(filename=filepath, width=4./3*360)

## Show time-dependent profile

In [ ]:
from functions import plot_omega_profile

In [ ]:
# parameters
dpi=150
fps=10
figsize=(6, 4.5)
lw=0.5
ms=4


mydir = figdir / 'profiles'
if not mydir.is_dir():
    mydir.mkdir(parents=True, exist_ok=True)

fpath = mydir / 'profile_T.mp4'
T_list = np.array([np.einsum('ta,a->ta', T_real[:,:],population), np.einsum('ta,a->ta', T_fit[:,:], population),  np.einsum('ta,a->ta', T[:,:], population)])
ylabel="$T_a$"
plot_omega_profile(T_list, times, labels=['real', 'fit', 'simplified'], colors=['red', 'darkblue', 'darkgreen'], \
                   fileout=fpath, tpdir=mydir / 'snapshots_T', dpi=dpi, fps=fps, figsize=figsize, ylabel=ylabel, \
                   lw=lw, ms=ms, styles=['o', '-', '-'], deletetp=False, exts=['.png','.svg'], ymin=1.)

fpath = mydir / 'profile_dT.mp4'
T_list = np.array([np.einsum('ta,a->ta', dT_real[:,:],population), np.einsum('ta,a->ta', dT_fit[:,:], population),  np.einsum('ta,a->ta', dT[:,:], population)])
ylabel="$dT_a$"
plot_omega_profile(T_list, times, labels=['real', 'fit', 'simplified'], colors=['red', 'darkblue', 'darkgreen'], \
                   fileout=fpath, tpdir=mydir / 'snapshots_dT', dpi=dpi, fps=fps, figsize=figsize, ylabel=ylabel, \
                   lw=lw, ms=ms, styles=['o', '-', '-'], deletetp=False, exts=['.png','.svg'], ymin=1.)

In [ ]:
fpath = figdir / 'profiles' / 'profile_T.mp4'
HTML("""
<video height="360" controls>
  <source src="{:s}" type="video/mp4">
</video>
""".format(str(fpath)))

In [ ]:
fpath = figdir / 'profiles' / 'profile_dT.mp4'
HTML("""
<video height="360" controls>
  <source src="{:s}" type="video/mp4">
</video>
""".format(str(fpath)))

## Show time-dependent map

In [ ]:
from functions import plot_omega_map

In [ ]:
# parameters
dpi=150
fps=10
figsize=(6, 4.5)
lw=0.5
ms=4
idump=1

mydir = figdir / 'maps'
if not mydir.is_dir():
    mydir.mkdir(parents=True, exist_ok=True)


fpath = mydir / 'map_T.mp4'
plot_omega_map(np.einsum('ta,a->ta', T, population), times, XY=clusters.loc[:, ['X', 'Y']].to_numpy().T, \
fileout=fpath, tpdir=mydir / 'snapshots_T', dpi=dpi, fps=fps, figsize=figsize, idump=idump, \
               clabel="$T$", vmin=1., deletetp=False, exts=['.png','.svg'])
    
fpath = mydir / 'map_dT.mp4'
plot_omega_map(np.einsum('ta,a->ta', dT, population), times, XY=clusters.loc[:, ['X', 'Y']].to_numpy().T, \
fileout=fpath, tpdir=mydir / 'snapshots_dT', dpi=dpi, fps=fps, figsize=figsize, idump=idump, \
               clabel="$dT$", vmin=1., deletetp=False, exts=['.png','.svg'])

In [ ]:
fpath = figdir / 'maps' / 'map_T.mp4'
HTML("""
<video height="360" controls>
  <source src="{:s}" type="video/mp4">
</video>
""".format(str(fpath)))

In [ ]:
fpath = figdir / 'maps' / 'map_dT.mp4'
HTML("""
<video height="360" controls>
  <source src="{:s}" type="video/mp4">
</video>
""".format(str(fpath)))